In [1]:
import scipy.io as sio
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
import math
import copy
from collections import OrderedDict

In [2]:

def torch_device():
    if torch.cuda.is_available():
        return torch.device('cuda:0')
    else:
        return torch.device('cpu')
    
torch_device = torch_device()

def shuffle_arrays(data, labels):
    indexes = [i for i in range(data.shape[0])]
    random.shuffle(indexes)
    return data[indexes], labels[indexes]
    

def read_data():
    """
    labels: 0, 1, 2, 3 corresponding to the original labels 1, 2, 3, 4
    """
    raw_data1 = sio.loadmat("S1.mat")
    raw_data2 = sio.loadmat("S2.mat")
    raw_labels1 = sio.loadmat("TrueLabelsS1.mat")['TrueLabelsS1']
    raw_labels2 = sio.loadmat("TrueLabelsS2.mat")['TrueLabelsS2']
    
    #print(raw_data1['test_data'].shape)
    
    training_set1 = torch.zeros((160, 400, 10), dtype=torch.float, device=torch_device)
    training_labels1 = torch.tensor([i//40 for i in range(160)], dtype=torch.long, device=torch_device)
    test_set1 = torch.tensor(raw_data1['test_data'], dtype=torch.float, device=torch_device)
    test_labels1 = torch.tensor(raw_labels1, dtype=torch.long, device=torch_device) - 1
    
    training_set2 = torch.zeros((160, 400, 10), dtype=torch.float, device=torch_device)
    training_labels2 = torch.tensor([i//40 for i in range(160)], dtype=torch.long, device=torch_device)
    test_set2 = torch.tensor(raw_data2['test_data'], dtype=torch.float, device=torch_device)
    test_labels2 = torch.tensor(raw_labels2, dtype=torch.long, device=torch_device) - 1

    
    for i in range(4):
        start = i*40
        end = start + 40
        training_set1[start:end, :, :] = \
            torch.tensor(raw_data1['training_data'][0, i], dtype=torch.float, device=torch_device)
        training_set2[start:end, :, :] = \
            torch.tensor(raw_data2['training_data'][0, i], dtype=torch.float, device=torch_device)
        
    #print(test_labels1.shape, test_labels2.shape)
    return *shuffle_arrays(torch.cat((training_set1, training_set2), 0), 
                            torch.cat((training_labels1, training_labels2), 0)),\
            *shuffle_arrays(torch.cat((test_set1, test_set2), 0),
                            torch.cat((test_labels1[0], test_labels2[0]), 0))


In [3]:
training_data, training_labels,test_data, test_labels = read_data()

### Bidirection LSTM with Attention

In [4]:
class LSTMNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, fc_hidden_size, num_outputs, seq_len, device=torch_device):
        super(LSTMNetwork, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.fc_hidden_size = fc_hidden_size
        self.device = device
        self.num_outputs = num_outputs
        self.seq_len = seq_len
        
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, 1, batch_first=True, bidirectional=True)
        
        self.attention_fc = nn.Linear(seq_len*hidden_size*2, seq_len)
        
        self.fc1 = nn.Linear(self.hidden_size*2, self.fc_hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(self.fc_hidden_size, self.num_outputs)
        self.relu2 = nn.ReLU()
    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), self.hidden_size, dtype=torch.float).to(self.device)
        c0 = torch.zeros(2, x.size(0), self.hidden_size, dtype=torch.float).to(self.device)
        
        lstm_out, (h1, c1) = self.lstm(x, (h0, c0))
        #print(out.shape)
        #find attention: N, L, 2H -> N, L
        out = self.attention_fc(lstm_out.reshape(lstm_out.shape[0], lstm_out.shape[1]*lstm_out.shape[2]))
        attention = torch.softmax(out, dim=1).reshape(out.shape[0], out.shape[1], 1)
        #attension layer: (N, L) * (N, L, 2H) -> N, 2H   
        #print(attention.shape, lstm_out.shape)
        out = torch.mul(attention, lstm_out).sum(axis=1)
        
        # N,2H -> N, hidden_size
        out = self.relu1(self.fc1(out))
        out = self.relu2(self.fc2(out))
        
        # N, num_outputs
        return out
        

In [5]:
def predict(model, data, labels, lamb=0.01, device=torch_device, pr=True):
    """
    data: N * L * input_size
    labels: 1 * N
    """
    loss_func = nn.CrossEntropyLoss()
    with torch.no_grad():   
        out = model(data)
        
        #print(out.shape, labels.shape)
        loss = loss_func(out, labels.reshape(-1))

        l2reg = torch.tensor(0, dtype=torch.float, device=device)
        for p in model.parameters():
            l2reg += torch.pow(p, 2).sum()
        loss = loss + lamb * l2reg            

        softmax = nn.Softmax(dim=1)
        _, y_hat = softmax(out).max(axis=1)
        correct = (y_hat == labels).sum().item()
        total = y_hat.shape[0]

        if pr:
            print("result:", y_hat)
            print("labels:", labels)
            print("Loss:", loss.item(), "Correct Rate:", correct/total)
    return correct/total

In [6]:
def train_model(data, labels, test_data, test_labels, epoch, hidden_size, fc_hidden_size, 
                num_outputs=4, seq_len=400, lr=0.0005, 
                lamb=0.01, batch_size=32, device=torch_device):
    """
    data: N * L * input_size
    labels: 1 * N
    """
    #print("============ hidden_size:", hidden_size, "fc_hidden_size:", fc_hidden_size, 
#           "learning rate:", lr, "lambda:", lamb, "batch size:", batch_size, 
#           "============")
    input_size = data.shape[2]
    num_classes = 4
    num_samples = data.shape[0]
    
    model = LSTMNetwork(input_size, hidden_size, fc_hidden_size, num_classes, 
                        seq_len, device=torch_device).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_func = nn.CrossEntropyLoss()
    
    num_batches = math.ceil(num_samples/batch_size)
    
    best_acc = 0
    best_state = None
    losses = []
    accs = []
    for e in range(epoch):
        epoch_correct = 0
        epoch_loss = 0
        for i in range(num_batches):
            start = i * batch_size
            end = (i+1) * batch_size
            if end > num_samples:
                end = num-samples
            
            batch_data = data[start:end]
            batch_labels = labels[start:end]
            
            #print(batch_data.shape)
            out = model(batch_data)
            #print(batch_labels)
            #print(out)
            loss = loss_func(out, batch_labels)

            l2reg = torch.tensor(0, dtype=torch.float, device=device)
            for p in model.parameters():
                l2reg += torch.pow(p, 2).sum()
            loss = loss + lamb * l2reg            
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            with torch.no_grad():
                softmax = nn.Softmax(dim=1)
                _, y_hat = softmax(out).max(axis=1)
                batch_correct = (y_hat == batch_labels).sum().item()
                epoch_correct += batch_correct
                batch_total = y_hat.shape[0]
                epoch_loss += loss.item()
#                 print("result:", y_hat)
#                 print("labels:", batch_labels)
#                 print("Epoch:", e, "Batch ", i, "Loss:", loss.item(), 
#                       "Correct Rate:", batch_correct/batch_total)
                test_accuracy = predict(model, test_data, test_labels, device=torch_device, pr=False)
                if test_accuracy > best_acc:
                    best_acc = test_accuracy
                    best_state = copy.deepcopy(model.state_dict())
        losses.append(epoch_loss/num_batches)
        accs.append(test_accuracy)
        print("Epoch:", e, "Avg_loss:", epoch_loss/num_batches, 
              "training accuracy:", epoch_correct/num_samples,
              "test accuracy:", test_accuracy)
    print("best accuracy:", best_acc)
    return model, best_state, best_acc, losses, accs


In [7]:
from itertools import product
def find_best_model():
    lrs = [0.0001, 0.0005]
    fc_hiddens = [16, 32]
    lambs = [0.0001, 0.001]
    batch_sizes = [16, 32, 64]

    best_states = None
    best_accuracy = 0
    best_model = None
    params = []
    losses = []
    accs = []
    
    for l, fch, lm, bs in product(lrs, fc_hiddens, lambs, batch_sizes):
        print("==========lr:", l, "lambda:", lm, "batch_size:", bs, "fc_hidden:", fch, "==========")
        params.append((l, fch, lm, bs))
        model, best_state, best_acc, loss, acc = train_model(training_data, training_labels,
                                                             test_data, test_labels, 2000, 128, fch,
                                                             lr=l, lamb=lm, batch_size=bs)
        losses.append(loss)
        accs.append(acc)
        print("Best accuracy:", best_acc)
        if best_acc > best_accuracy:
            best_accuracy = best_acc
            best_states = best_state
            model.load_state_dict(best_states)
            best_model = model
    print("best overall accuracy:", best_accuracy)
    return best_model, params, losses, accs

#model, params, losses, accs = find_best_model()

model, best_state, best_acc, loss, acc = train_model(training_data, training_labels,
                                                             test_data, test_labels, 2000, 128, 16,
                                                             lr=0.0001, lamb=0.001, batch_size=16)

Epoch: 0 Avg_loss: 1.858587634563446 training accuracy: 0.25 test accuracy: 0.2653061224489796
Epoch: 1 Avg_loss: 1.7489727795124055 training accuracy: 0.25625 test accuracy: 0.2653061224489796
Epoch: 2 Avg_loss: 1.6701273560523986 training accuracy: 0.39375 test accuracy: 0.25170068027210885
Epoch: 3 Avg_loss: 1.6119879364967347 training accuracy: 0.490625 test accuracy: 0.2653061224489796
Epoch: 4 Avg_loss: 1.5592638790607452 training accuracy: 0.55625 test accuracy: 0.25170068027210885
Epoch: 5 Avg_loss: 1.5082694053649903 training accuracy: 0.58125 test accuracy: 0.25170068027210885
Epoch: 6 Avg_loss: 1.4538520276546478 training accuracy: 0.615625 test accuracy: 0.21768707482993196
Epoch: 7 Avg_loss: 1.4029042303562165 training accuracy: 0.625 test accuracy: 0.23129251700680273
Epoch: 8 Avg_loss: 1.3514792442321777 training accuracy: 0.640625 test accuracy: 0.23129251700680273
Epoch: 9 Avg_loss: 1.2991986274719238 training accuracy: 0.64375 test accuracy: 0.24489795918367346
Epoch:

Epoch: 83 Avg_loss: 0.1291828092187643 training accuracy: 1.0 test accuracy: 0.23129251700680273
Epoch: 84 Avg_loss: 0.12712659277021884 training accuracy: 1.0 test accuracy: 0.23129251700680273
Epoch: 85 Avg_loss: 0.12515566423535346 training accuracy: 1.0 test accuracy: 0.23809523809523808
Epoch: 86 Avg_loss: 0.12326503060758114 training accuracy: 1.0 test accuracy: 0.23129251700680273
Epoch: 87 Avg_loss: 0.1214447233825922 training accuracy: 1.0 test accuracy: 0.23809523809523808
Epoch: 88 Avg_loss: 0.11964516900479794 training accuracy: 1.0 test accuracy: 0.23809523809523808
Epoch: 89 Avg_loss: 0.11787703596055507 training accuracy: 1.0 test accuracy: 0.23809523809523808
Epoch: 90 Avg_loss: 0.1162541825324297 training accuracy: 1.0 test accuracy: 0.23809523809523808
Epoch: 91 Avg_loss: 0.11641581542789936 training accuracy: 1.0 test accuracy: 0.2653061224489796
Epoch: 92 Avg_loss: 0.11350857056677341 training accuracy: 1.0 test accuracy: 0.2585034013605442
Epoch: 93 Avg_loss: 0.111

Epoch: 167 Avg_loss: 0.06357639599591494 training accuracy: 1.0 test accuracy: 0.272108843537415
Epoch: 168 Avg_loss: 0.06323468424379826 training accuracy: 1.0 test accuracy: 0.2789115646258503
Epoch: 169 Avg_loss: 0.0628777852281928 training accuracy: 1.0 test accuracy: 0.272108843537415
Epoch: 170 Avg_loss: 0.06257536709308624 training accuracy: 1.0 test accuracy: 0.2789115646258503
Epoch: 171 Avg_loss: 0.06227417271584272 training accuracy: 1.0 test accuracy: 0.272108843537415
Epoch: 172 Avg_loss: 0.06198471058160067 training accuracy: 1.0 test accuracy: 0.2789115646258503
Epoch: 173 Avg_loss: 0.06174509860575199 training accuracy: 1.0 test accuracy: 0.272108843537415
Epoch: 174 Avg_loss: 0.06145312562584877 training accuracy: 1.0 test accuracy: 0.272108843537415
Epoch: 175 Avg_loss: 0.061181171983480456 training accuracy: 1.0 test accuracy: 0.272108843537415
Epoch: 176 Avg_loss: 0.06092646811157465 training accuracy: 1.0 test accuracy: 0.272108843537415
Epoch: 177 Avg_loss: 0.0607

Epoch: 251 Avg_loss: 0.3558800175786018 training accuracy: 0.95 test accuracy: 0.24489795918367346
Epoch: 252 Avg_loss: 0.33559834137558936 training accuracy: 0.953125 test accuracy: 0.24489795918367346
Epoch: 253 Avg_loss: 0.3170922093093395 training accuracy: 0.959375 test accuracy: 0.24489795918367346
Epoch: 254 Avg_loss: 0.3020668588578701 training accuracy: 0.96875 test accuracy: 0.24489795918367346
Epoch: 255 Avg_loss: 0.28984596505761145 training accuracy: 0.96875 test accuracy: 0.2585034013605442
Epoch: 256 Avg_loss: 0.27894559502601624 training accuracy: 0.96875 test accuracy: 0.25170068027210885
Epoch: 257 Avg_loss: 0.2700342804193497 training accuracy: 0.96875 test accuracy: 0.2585034013605442
Epoch: 258 Avg_loss: 0.2620001696050167 training accuracy: 0.96875 test accuracy: 0.2653061224489796
Epoch: 259 Avg_loss: 0.2531756035983562 training accuracy: 0.96875 test accuracy: 0.2653061224489796
Epoch: 260 Avg_loss: 0.24538801684975625 training accuracy: 0.96875 test accuracy: 0

Epoch: 332 Avg_loss: 0.09361046701669692 training accuracy: 0.996875 test accuracy: 0.2585034013605442
Epoch: 333 Avg_loss: 0.09299983382225037 training accuracy: 0.996875 test accuracy: 0.2585034013605442
Epoch: 334 Avg_loss: 0.09248368889093399 training accuracy: 0.996875 test accuracy: 0.2585034013605442
Epoch: 335 Avg_loss: 0.09199794717133045 training accuracy: 0.996875 test accuracy: 0.2585034013605442
Epoch: 336 Avg_loss: 0.09149864353239537 training accuracy: 0.996875 test accuracy: 0.2585034013605442
Epoch: 337 Avg_loss: 0.09101678058505058 training accuracy: 0.996875 test accuracy: 0.2585034013605442
Epoch: 338 Avg_loss: 0.09055250845849513 training accuracy: 0.996875 test accuracy: 0.2585034013605442
Epoch: 339 Avg_loss: 0.09008402191102505 training accuracy: 0.996875 test accuracy: 0.25170068027210885
Epoch: 340 Avg_loss: 0.08964115753769875 training accuracy: 0.996875 test accuracy: 0.25170068027210885
Epoch: 341 Avg_loss: 0.08920563198626041 training accuracy: 0.996875 te

...

Epoch: 1654 Avg_loss: 0.027248430345207453 training accuracy: 1.0 test accuracy: 0.25170068027210885
Epoch: 1655 Avg_loss: 0.027190298400819303 training accuracy: 1.0 test accuracy: 0.25170068027210885
Epoch: 1656 Avg_loss: 0.02709284219890833 training accuracy: 1.0 test accuracy: 0.24489795918367346
Epoch: 1657 Avg_loss: 0.027108015585690738 training accuracy: 1.0 test accuracy: 0.24489795918367346
Epoch: 1658 Avg_loss: 0.028325047064572572 training accuracy: 1.0 test accuracy: 0.2653061224489796
Epoch: 1659 Avg_loss: 0.11620278172194957 training accuracy: 0.98125 test accuracy: 0.2585034013605442
Epoch: 1660 Avg_loss: 0.24944529458880424 training accuracy: 0.953125 test accuracy: 0.19727891156462585
Epoch: 1661 Avg_loss: 0.35125477612018585 training accuracy: 0.91875 test accuracy: 0.2653061224489796
Epoch: 1662 Avg_loss: 0.22473815679550171 training accuracy: 0.978125 test accuracy: 0.25170068027210885
Epoch: 1663 Avg_loss: 0.15633589997887612 training accuracy: 0.996875 test accura

Epoch: 1736 Avg_loss: 0.04582353699952364 training accuracy: 1.0 test accuracy: 0.2585034013605442
Epoch: 1737 Avg_loss: 0.04547926690429449 training accuracy: 1.0 test accuracy: 0.2585034013605442
Epoch: 1738 Avg_loss: 0.04514094740152359 training accuracy: 1.0 test accuracy: 0.25170068027210885
Epoch: 1739 Avg_loss: 0.044808370247483256 training accuracy: 1.0 test accuracy: 0.25170068027210885
Epoch: 1740 Avg_loss: 0.04447731655091047 training accuracy: 1.0 test accuracy: 0.25170068027210885
Epoch: 1741 Avg_loss: 0.04414577968418598 training accuracy: 1.0 test accuracy: 0.25170068027210885
Epoch: 1742 Avg_loss: 0.04380885362625122 training accuracy: 1.0 test accuracy: 0.2585034013605442
Epoch: 1743 Avg_loss: 0.04349514674395323 training accuracy: 1.0 test accuracy: 0.272108843537415
Epoch: 1744 Avg_loss: 0.043218493834137915 training accuracy: 1.0 test accuracy: 0.2789115646258503
Epoch: 1745 Avg_loss: 0.04290646854788065 training accuracy: 1.0 test accuracy: 0.2789115646258503
Epoch

Epoch: 1819 Avg_loss: 0.16075260527431964 training accuracy: 0.96875 test accuracy: 0.21768707482993196
Epoch: 1820 Avg_loss: 0.4019081130623817 training accuracy: 0.903125 test accuracy: 0.21768707482993196
Epoch: 1821 Avg_loss: 0.38221232816576955 training accuracy: 0.9375 test accuracy: 0.2653061224489796
Epoch: 1822 Avg_loss: 0.30356378853321075 training accuracy: 0.965625 test accuracy: 0.23809523809523808
Epoch: 1823 Avg_loss: 0.22309357449412345 training accuracy: 0.990625 test accuracy: 0.2653061224489796
Epoch: 1824 Avg_loss: 0.18267636597156525 training accuracy: 0.996875 test accuracy: 0.24489795918367346
Epoch: 1825 Avg_loss: 0.16275155618786813 training accuracy: 1.0 test accuracy: 0.2653061224489796
Epoch: 1826 Avg_loss: 0.15200342163443564 training accuracy: 1.0 test accuracy: 0.272108843537415
Epoch: 1827 Avg_loss: 0.14445050433278084 training accuracy: 1.0 test accuracy: 0.2789115646258503
Epoch: 1828 Avg_loss: 0.1385899968445301 training accuracy: 1.0 test accuracy: 0

Epoch: 1902 Avg_loss: 0.05515629705041647 training accuracy: 1.0 test accuracy: 0.23129251700680273
Epoch: 1903 Avg_loss: 0.054730449058115485 training accuracy: 1.0 test accuracy: 0.23809523809523808
Epoch: 1904 Avg_loss: 0.054334111511707306 training accuracy: 1.0 test accuracy: 0.23129251700680273
Epoch: 1905 Avg_loss: 0.05393608808517456 training accuracy: 1.0 test accuracy: 0.23809523809523808
Epoch: 1906 Avg_loss: 0.05348636955022812 training accuracy: 1.0 test accuracy: 0.23809523809523808
Epoch: 1907 Avg_loss: 0.05302883237600327 training accuracy: 1.0 test accuracy: 0.23809523809523808
Epoch: 1908 Avg_loss: 0.05263795889914036 training accuracy: 1.0 test accuracy: 0.23809523809523808
Epoch: 1909 Avg_loss: 0.05229272600263357 training accuracy: 1.0 test accuracy: 0.23809523809523808
Epoch: 1910 Avg_loss: 0.05190126076340675 training accuracy: 1.0 test accuracy: 0.24489795918367346
Epoch: 1911 Avg_loss: 0.05152410306036472 training accuracy: 1.0 test accuracy: 0.2380952380952380

Epoch: 1985 Avg_loss: 0.03228980824351311 training accuracy: 1.0 test accuracy: 0.2857142857142857
Epoch: 1986 Avg_loss: 0.032087853737175466 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 1987 Avg_loss: 0.03193140774965286 training accuracy: 1.0 test accuracy: 0.2857142857142857
Epoch: 1988 Avg_loss: 0.03178709503263235 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 1989 Avg_loss: 0.03162528425455093 training accuracy: 1.0 test accuracy: 0.2857142857142857
Epoch: 1990 Avg_loss: 0.03146915957331657 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 1991 Avg_loss: 0.0313377040438354 training accuracy: 1.0 test accuracy: 0.3197278911564626
Epoch: 1992 Avg_loss: 0.031238813232630492 training accuracy: 1.0 test accuracy: 0.2857142857142857
Epoch: 1993 Avg_loss: 0.031032060831785203 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 1994 Avg_loss: 0.030853752978146076 training accuracy: 1.0 test accuracy: 0.29931972789115646
Epoch:

In [8]:
model1, best_state1, best_acc1, loss1, acc1 = train_model(training_data, training_labels,
                                                             test_data, test_labels, 2000, 128, 16,
                                                             lr=0.0005, lamb=0.001, batch_size=16)

Epoch: 0 Avg_loss: 1.7942642331123353 training accuracy: 0.25625 test accuracy: 0.30612244897959184
Epoch: 1 Avg_loss: 1.6364943265914917 training accuracy: 0.3375 test accuracy: 0.29931972789115646
Epoch: 2 Avg_loss: 1.5354824185371398 training accuracy: 0.4125 test accuracy: 0.32653061224489793
Epoch: 3 Avg_loss: 1.4539529979228973 training accuracy: 0.44375 test accuracy: 0.29931972789115646
Epoch: 4 Avg_loss: 1.3886882781982421 training accuracy: 0.46875 test accuracy: 0.29931972789115646
Epoch: 5 Avg_loss: 1.327773141860962 training accuracy: 0.4875 test accuracy: 0.2857142857142857
Epoch: 6 Avg_loss: 1.270124888420105 training accuracy: 0.490625 test accuracy: 0.30612244897959184
Epoch: 7 Avg_loss: 1.2129532098770142 training accuracy: 0.503125 test accuracy: 0.3197278911564626
Epoch: 8 Avg_loss: 1.1593836545944214 training accuracy: 0.50625 test accuracy: 0.29931972789115646
Epoch: 9 Avg_loss: 1.1085134774446488 training accuracy: 0.53125 test accuracy: 0.29931972789115646
Epoch

Epoch: 84 Avg_loss: 0.07351240888237953 training accuracy: 1.0 test accuracy: 0.3197278911564626
Epoch: 85 Avg_loss: 0.07287485003471375 training accuracy: 1.0 test accuracy: 0.3197278911564626
Epoch: 86 Avg_loss: 0.07225903421640396 training accuracy: 1.0 test accuracy: 0.3197278911564626
Epoch: 87 Avg_loss: 0.07165525108575821 training accuracy: 1.0 test accuracy: 0.3197278911564626
Epoch: 88 Avg_loss: 0.07106685899198055 training accuracy: 1.0 test accuracy: 0.3197278911564626
Epoch: 89 Avg_loss: 0.07049081884324551 training accuracy: 1.0 test accuracy: 0.3197278911564626
Epoch: 90 Avg_loss: 0.06993718519806862 training accuracy: 1.0 test accuracy: 0.32653061224489793
Epoch: 91 Avg_loss: 0.06939276829361915 training accuracy: 1.0 test accuracy: 0.32653061224489793
Epoch: 92 Avg_loss: 0.06885557807981968 training accuracy: 1.0 test accuracy: 0.32653061224489793
Epoch: 93 Avg_loss: 0.06833639033138753 training accuracy: 1.0 test accuracy: 0.32653061224489793
Epoch: 94 Avg_loss: 0.0678

Epoch: 168 Avg_loss: 0.4237556427717209 training accuracy: 0.96875 test accuracy: 0.2653061224489796
Epoch: 169 Avg_loss: 0.4204917848110199 training accuracy: 0.965625 test accuracy: 0.30612244897959184
Epoch: 170 Avg_loss: 0.4270595222711563 training accuracy: 0.96875 test accuracy: 0.2653061224489796
Epoch: 171 Avg_loss: 0.3845493882894516 training accuracy: 0.978125 test accuracy: 0.30612244897959184
Epoch: 172 Avg_loss: 0.30544166937470435 training accuracy: 0.984375 test accuracy: 0.272108843537415
Epoch: 173 Avg_loss: 0.26240299046039584 training accuracy: 0.996875 test accuracy: 0.2857142857142857
Epoch: 174 Avg_loss: 0.24031824991106987 training accuracy: 0.996875 test accuracy: 0.30612244897959184
Epoch: 175 Avg_loss: 0.22499456405639648 training accuracy: 0.996875 test accuracy: 0.30612244897959184
Epoch: 176 Avg_loss: 0.21158422753214837 training accuracy: 1.0 test accuracy: 0.29931972789115646
Epoch: 177 Avg_loss: 0.20239376202225684 training accuracy: 1.0 test accuracy: 0

Epoch: 251 Avg_loss: 0.07245969921350479 training accuracy: 1.0 test accuracy: 0.3197278911564626
Epoch: 252 Avg_loss: 0.07188420295715332 training accuracy: 1.0 test accuracy: 0.3197278911564626
Epoch: 253 Avg_loss: 0.07132659927010536 training accuracy: 1.0 test accuracy: 0.3197278911564626
Epoch: 254 Avg_loss: 0.07077444382011891 training accuracy: 1.0 test accuracy: 0.3197278911564626
Epoch: 255 Avg_loss: 0.07023925557732583 training accuracy: 1.0 test accuracy: 0.3197278911564626
Epoch: 256 Avg_loss: 0.06971255913376809 training accuracy: 1.0 test accuracy: 0.32653061224489793
Epoch: 257 Avg_loss: 0.06919490061700344 training accuracy: 1.0 test accuracy: 0.32653061224489793
Epoch: 258 Avg_loss: 0.06868972517549991 training accuracy: 1.0 test accuracy: 0.32653061224489793
Epoch: 259 Avg_loss: 0.06819362267851829 training accuracy: 1.0 test accuracy: 0.32653061224489793
Epoch: 260 Avg_loss: 0.06771055683493614 training accuracy: 1.0 test accuracy: 0.32653061224489793
Epoch: 261 Avg_

Epoch: 335 Avg_loss: 0.04887810982763767 training accuracy: 1.0 test accuracy: 0.3129251700680272
Epoch: 336 Avg_loss: 0.048740383423864844 training accuracy: 1.0 test accuracy: 0.3129251700680272
Epoch: 337 Avg_loss: 0.048611432500183585 training accuracy: 1.0 test accuracy: 0.3197278911564626
Epoch: 338 Avg_loss: 0.04848599229007959 training accuracy: 1.0 test accuracy: 0.3129251700680272
Epoch: 339 Avg_loss: 0.04836405087262392 training accuracy: 1.0 test accuracy: 0.3129251700680272
Epoch: 340 Avg_loss: 0.04823785647749901 training accuracy: 1.0 test accuracy: 0.3197278911564626
Epoch: 341 Avg_loss: 0.04812504798173904 training accuracy: 1.0 test accuracy: 0.30612244897959184
Epoch: 342 Avg_loss: 0.047992178797721864 training accuracy: 1.0 test accuracy: 0.32653061224489793
Epoch: 343 Avg_loss: 0.04784774500876665 training accuracy: 1.0 test accuracy: 0.32653061224489793
Epoch: 344 Avg_loss: 0.047726256959140304 training accuracy: 1.0 test accuracy: 0.3333333333333333
Epoch: 345 Av

...

Epoch: 1662 Avg_loss: 0.08271019384264947 training accuracy: 1.0 test accuracy: 0.16326530612244897
Epoch: 1663 Avg_loss: 0.0803349707275629 training accuracy: 1.0 test accuracy: 0.16326530612244897
Epoch: 1664 Avg_loss: 0.07812321484088898 training accuracy: 1.0 test accuracy: 0.1564625850340136
Epoch: 1665 Avg_loss: 0.07606885060667992 training accuracy: 1.0 test accuracy: 0.1564625850340136
Epoch: 1666 Avg_loss: 0.07415246181190013 training accuracy: 1.0 test accuracy: 0.1564625850340136
Epoch: 1667 Avg_loss: 0.07235762178897857 training accuracy: 1.0 test accuracy: 0.1564625850340136
Epoch: 1668 Avg_loss: 0.07067667171359063 training accuracy: 1.0 test accuracy: 0.1564625850340136
Epoch: 1669 Avg_loss: 0.06909920200705529 training accuracy: 1.0 test accuracy: 0.1564625850340136
Epoch: 1670 Avg_loss: 0.0676219467073679 training accuracy: 1.0 test accuracy: 0.1564625850340136
Epoch: 1671 Avg_loss: 0.066232605651021 training accuracy: 1.0 test accuracy: 0.1564625850340136
Epoch: 1672 

Epoch: 1744 Avg_loss: 0.03835594989359379 training accuracy: 1.0 test accuracy: 0.21768707482993196
Epoch: 1745 Avg_loss: 0.03825625944882631 training accuracy: 1.0 test accuracy: 0.21768707482993196
Epoch: 1746 Avg_loss: 0.03815676122903824 training accuracy: 1.0 test accuracy: 0.21768707482993196
Epoch: 1747 Avg_loss: 0.03806433472782374 training accuracy: 1.0 test accuracy: 0.21768707482993196
Epoch: 1748 Avg_loss: 0.037966891378164294 training accuracy: 1.0 test accuracy: 0.22448979591836735
Epoch: 1749 Avg_loss: 0.037876509688794614 training accuracy: 1.0 test accuracy: 0.22448979591836735
Epoch: 1750 Avg_loss: 0.03778420574963093 training accuracy: 1.0 test accuracy: 0.22448979591836735
Epoch: 1751 Avg_loss: 0.03769659698009491 training accuracy: 1.0 test accuracy: 0.22448979591836735
Epoch: 1752 Avg_loss: 0.0376033328473568 training accuracy: 1.0 test accuracy: 0.22448979591836735
Epoch: 1753 Avg_loss: 0.037511407397687435 training accuracy: 1.0 test accuracy: 0.2244897959183673

Epoch: 1826 Avg_loss: 0.03252746947109699 training accuracy: 1.0 test accuracy: 0.2108843537414966
Epoch: 1827 Avg_loss: 0.03246357515454292 training accuracy: 1.0 test accuracy: 0.2108843537414966
Epoch: 1828 Avg_loss: 0.032427207380533216 training accuracy: 1.0 test accuracy: 0.2108843537414966
Epoch: 1829 Avg_loss: 0.03234769683331251 training accuracy: 1.0 test accuracy: 0.2108843537414966
Epoch: 1830 Avg_loss: 0.03232666403055191 training accuracy: 1.0 test accuracy: 0.2108843537414966
Epoch: 1831 Avg_loss: 0.03227154687047005 training accuracy: 1.0 test accuracy: 0.2108843537414966
Epoch: 1832 Avg_loss: 0.03222227301448584 training accuracy: 1.0 test accuracy: 0.2108843537414966
Epoch: 1833 Avg_loss: 0.03214567303657532 training accuracy: 1.0 test accuracy: 0.2108843537414966
Epoch: 1834 Avg_loss: 0.03211615215986967 training accuracy: 1.0 test accuracy: 0.2108843537414966
Epoch: 1835 Avg_loss: 0.03203937914222479 training accuracy: 1.0 test accuracy: 0.2108843537414966
Epoch: 18

Epoch: 1908 Avg_loss: 0.029055984038859606 training accuracy: 1.0 test accuracy: 0.20408163265306123
Epoch: 1909 Avg_loss: 0.028983157966285943 training accuracy: 1.0 test accuracy: 0.2108843537414966
Epoch: 1910 Avg_loss: 0.029006305430084468 training accuracy: 1.0 test accuracy: 0.2108843537414966
Epoch: 1911 Avg_loss: 0.028916120063513517 training accuracy: 1.0 test accuracy: 0.20408163265306123
Epoch: 1912 Avg_loss: 0.0289141315035522 training accuracy: 1.0 test accuracy: 0.2108843537414966
Epoch: 1913 Avg_loss: 0.028893814142793416 training accuracy: 1.0 test accuracy: 0.21768707482993196
Epoch: 1914 Avg_loss: 0.02892464930191636 training accuracy: 1.0 test accuracy: 0.2108843537414966
Epoch: 1915 Avg_loss: 0.028848292399197818 training accuracy: 1.0 test accuracy: 0.22448979591836735
Epoch: 1916 Avg_loss: 0.02884596083313227 training accuracy: 1.0 test accuracy: 0.2108843537414966
Epoch: 1917 Avg_loss: 0.02886961530894041 training accuracy: 1.0 test accuracy: 0.2108843537414966
E

Epoch: 1990 Avg_loss: 0.02719611907377839 training accuracy: 1.0 test accuracy: 0.1836734693877551
Epoch: 1991 Avg_loss: 0.027220007963478565 training accuracy: 1.0 test accuracy: 0.19727891156462585
Epoch: 1992 Avg_loss: 0.02723157647997141 training accuracy: 1.0 test accuracy: 0.20408163265306123
Epoch: 1993 Avg_loss: 0.027241753228008748 training accuracy: 1.0 test accuracy: 0.19727891156462585
Epoch: 1994 Avg_loss: 0.027222456969320775 training accuracy: 1.0 test accuracy: 0.19047619047619047
Epoch: 1995 Avg_loss: 0.02721007913351059 training accuracy: 1.0 test accuracy: 0.2108843537414966
Epoch: 1996 Avg_loss: 0.027223102375864984 training accuracy: 1.0 test accuracy: 0.20408163265306123
Epoch: 1997 Avg_loss: 0.027059775218367575 training accuracy: 1.0 test accuracy: 0.1836734693877551
Epoch: 1998 Avg_loss: 0.026998469419777394 training accuracy: 1.0 test accuracy: 0.17687074829931973
Epoch: 1999 Avg_loss: 0.02695232620462775 training accuracy: 1.0 test accuracy: 0.190476190476190

In [9]:
model1, best_state1, best_acc1, loss1, acc1 = train_model(training_data, training_labels,
                                                             test_data, test_labels, 2000, 128, 32,
                                                             lr=0.0001, lamb=0.001, batch_size=16)

Epoch: 0 Avg_loss: 1.8756811320781708 training accuracy: 0.25625 test accuracy: 0.21768707482993196
Epoch: 1 Avg_loss: 1.7372596561908722 training accuracy: 0.575 test accuracy: 0.24489795918367346
Epoch: 2 Avg_loss: 1.612297385931015 training accuracy: 0.690625 test accuracy: 0.22448979591836735
Epoch: 3 Avg_loss: 1.5158017098903656 training accuracy: 0.709375 test accuracy: 0.23129251700680273
Epoch: 4 Avg_loss: 1.4256671905517577 training accuracy: 0.753125 test accuracy: 0.24489795918367346
Epoch: 5 Avg_loss: 1.3331609308719634 training accuracy: 0.809375 test accuracy: 0.23129251700680273
Epoch: 6 Avg_loss: 1.2446162164211274 training accuracy: 0.853125 test accuracy: 0.3129251700680272
Epoch: 7 Avg_loss: 1.1572196424007415 training accuracy: 0.8875 test accuracy: 0.29931972789115646
Epoch: 8 Avg_loss: 1.0777524650096892 training accuracy: 0.90625 test accuracy: 0.2653061224489796
Epoch: 9 Avg_loss: 1.000350546836853 training accuracy: 0.915625 test accuracy: 0.25170068027210885
E

Epoch: 81 Avg_loss: 0.10737916082143784 training accuracy: 0.99375 test accuracy: 0.21768707482993196
Epoch: 82 Avg_loss: 0.10600860975682735 training accuracy: 0.99375 test accuracy: 0.21768707482993196
Epoch: 83 Avg_loss: 0.10478695444762706 training accuracy: 0.996875 test accuracy: 0.21768707482993196
Epoch: 84 Avg_loss: 0.10366972871124744 training accuracy: 0.996875 test accuracy: 0.2108843537414966
Epoch: 85 Avg_loss: 0.10256454683840274 training accuracy: 0.996875 test accuracy: 0.2108843537414966
Epoch: 86 Avg_loss: 0.10146630965173245 training accuracy: 0.996875 test accuracy: 0.2108843537414966
Epoch: 87 Avg_loss: 0.10042942985892296 training accuracy: 0.996875 test accuracy: 0.20408163265306123
Epoch: 88 Avg_loss: 0.0994350142776966 training accuracy: 0.996875 test accuracy: 0.20408163265306123
Epoch: 89 Avg_loss: 0.09848556742072105 training accuracy: 0.996875 test accuracy: 0.2108843537414966
Epoch: 90 Avg_loss: 0.0975714385509491 training accuracy: 0.996875 test accuracy

Epoch: 162 Avg_loss: 0.059104386158287524 training accuracy: 1.0 test accuracy: 0.1836734693877551
Epoch: 163 Avg_loss: 0.058815358020365235 training accuracy: 1.0 test accuracy: 0.19727891156462585
Epoch: 164 Avg_loss: 0.05860473122447729 training accuracy: 1.0 test accuracy: 0.19047619047619047
Epoch: 165 Avg_loss: 0.058307772129774095 training accuracy: 1.0 test accuracy: 0.1836734693877551
Epoch: 166 Avg_loss: 0.05796826537698507 training accuracy: 1.0 test accuracy: 0.1836734693877551
Epoch: 167 Avg_loss: 0.05764123108237982 training accuracy: 1.0 test accuracy: 0.19047619047619047
Epoch: 168 Avg_loss: 0.05741295684129 training accuracy: 1.0 test accuracy: 0.1836734693877551
Epoch: 169 Avg_loss: 0.05710618048906326 training accuracy: 1.0 test accuracy: 0.17687074829931973
Epoch: 170 Avg_loss: 0.05682930033653975 training accuracy: 1.0 test accuracy: 0.19727891156462585
Epoch: 171 Avg_loss: 0.05653491150587797 training accuracy: 1.0 test accuracy: 0.17006802721088435
Epoch: 172 Avg

Epoch: 244 Avg_loss: 0.09174788929522038 training accuracy: 1.0 test accuracy: 0.2585034013605442
Epoch: 245 Avg_loss: 0.091060084477067 training accuracy: 1.0 test accuracy: 0.2585034013605442
Epoch: 246 Avg_loss: 0.09039693623781205 training accuracy: 1.0 test accuracy: 0.2585034013605442
Epoch: 247 Avg_loss: 0.08972865790128708 training accuracy: 1.0 test accuracy: 0.2585034013605442
Epoch: 248 Avg_loss: 0.08908019624650479 training accuracy: 1.0 test accuracy: 0.2653061224489796
Epoch: 249 Avg_loss: 0.08843643181025981 training accuracy: 1.0 test accuracy: 0.2653061224489796
Epoch: 250 Avg_loss: 0.08779551684856415 training accuracy: 1.0 test accuracy: 0.2653061224489796
Epoch: 251 Avg_loss: 0.08716648556292057 training accuracy: 1.0 test accuracy: 0.2653061224489796
Epoch: 252 Avg_loss: 0.08653887249529361 training accuracy: 1.0 test accuracy: 0.2653061224489796
Epoch: 253 Avg_loss: 0.08591564744710922 training accuracy: 1.0 test accuracy: 0.2653061224489796
Epoch: 254 Avg_loss: 0

Epoch: 328 Avg_loss: 0.051096192188560964 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 329 Avg_loss: 0.050761725194752215 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 330 Avg_loss: 0.05042886901646852 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 331 Avg_loss: 0.05011011976748705 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 332 Avg_loss: 0.04979390166699886 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 333 Avg_loss: 0.049473806843161584 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 334 Avg_loss: 0.04915944244712591 training accuracy: 1.0 test accuracy: 0.2857142857142857
Epoch: 335 Avg_loss: 0.048861789889633654 training accuracy: 1.0 test accuracy: 0.2857142857142857
Epoch: 336 Avg_loss: 0.0485792625695467 training accuracy: 1.0 test accuracy: 0.2857142857142857
Epoch: 337 Avg_loss: 0.04825112074613571 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 338 Avg_lo

...

Epoch: 1651 Avg_loss: 0.031939410511404275 training accuracy: 1.0 test accuracy: 0.23809523809523808
Epoch: 1652 Avg_loss: 0.03167398180812597 training accuracy: 1.0 test accuracy: 0.23809523809523808
Epoch: 1653 Avg_loss: 0.031479418836534025 training accuracy: 1.0 test accuracy: 0.21768707482993196
Epoch: 1654 Avg_loss: 0.03134275432676077 training accuracy: 1.0 test accuracy: 0.2108843537414966
Epoch: 1655 Avg_loss: 0.031116601079702377 training accuracy: 1.0 test accuracy: 0.23129251700680273
Epoch: 1656 Avg_loss: 0.0308811504393816 training accuracy: 1.0 test accuracy: 0.22448979591836735
Epoch: 1657 Avg_loss: 0.030685665551573038 training accuracy: 1.0 test accuracy: 0.23129251700680273
Epoch: 1658 Avg_loss: 0.030503298062831162 training accuracy: 1.0 test accuracy: 0.22448979591836735
Epoch: 1659 Avg_loss: 0.030317553877830507 training accuracy: 1.0 test accuracy: 0.24489795918367346
Epoch: 1660 Avg_loss: 0.030123440362513066 training accuracy: 1.0 test accuracy: 0.2312925170068

Epoch: 1732 Avg_loss: 0.06997505202889442 training accuracy: 0.996875 test accuracy: 0.23129251700680273
Epoch: 1733 Avg_loss: 0.06902136243879795 training accuracy: 1.0 test accuracy: 0.23129251700680273
Epoch: 1734 Avg_loss: 0.06810387782752514 training accuracy: 1.0 test accuracy: 0.23129251700680273
Epoch: 1735 Avg_loss: 0.06722925696521997 training accuracy: 1.0 test accuracy: 0.23129251700680273
Epoch: 1736 Avg_loss: 0.0663486609235406 training accuracy: 1.0 test accuracy: 0.23129251700680273
Epoch: 1737 Avg_loss: 0.06548385210335254 training accuracy: 1.0 test accuracy: 0.22448979591836735
Epoch: 1738 Avg_loss: 0.06465674806386232 training accuracy: 1.0 test accuracy: 0.22448979591836735
Epoch: 1739 Avg_loss: 0.06387150343507528 training accuracy: 1.0 test accuracy: 0.23809523809523808
Epoch: 1740 Avg_loss: 0.06309702899307013 training accuracy: 1.0 test accuracy: 0.25170068027210885
Epoch: 1741 Avg_loss: 0.062358960136771203 training accuracy: 1.0 test accuracy: 0.2517006802721

Epoch: 1815 Avg_loss: 0.03014680165797472 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 1816 Avg_loss: 0.030040144082158805 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 1817 Avg_loss: 0.029736494272947313 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 1818 Avg_loss: 0.029484369978308677 training accuracy: 1.0 test accuracy: 0.2857142857142857
Epoch: 1819 Avg_loss: 0.029327946621924637 training accuracy: 1.0 test accuracy: 0.2857142857142857
Epoch: 1820 Avg_loss: 0.029154849890619516 training accuracy: 1.0 test accuracy: 0.2789115646258503
Epoch: 1821 Avg_loss: 0.028974893130362035 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 1822 Avg_loss: 0.02877056086435914 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 1823 Avg_loss: 0.028640538174659013 training accuracy: 1.0 test accuracy: 0.272108843537415
Epoch: 1824 Avg_loss: 0.028477213252335785 training accuracy: 1.0 test accuracy: 0.2857142857142857
Epo

Epoch: 1898 Avg_loss: 0.06432375386357307 training accuracy: 1.0 test accuracy: 0.2789115646258503
Epoch: 1899 Avg_loss: 0.06364089492708444 training accuracy: 1.0 test accuracy: 0.2789115646258503
Epoch: 1900 Avg_loss: 0.06295102182775736 training accuracy: 1.0 test accuracy: 0.2789115646258503
Epoch: 1901 Avg_loss: 0.06227812506258488 training accuracy: 1.0 test accuracy: 0.2789115646258503
Epoch: 1902 Avg_loss: 0.061609435081481936 training accuracy: 1.0 test accuracy: 0.2789115646258503
Epoch: 1903 Avg_loss: 0.06096287909895182 training accuracy: 1.0 test accuracy: 0.2857142857142857
Epoch: 1904 Avg_loss: 0.06032835654914379 training accuracy: 1.0 test accuracy: 0.2857142857142857
Epoch: 1905 Avg_loss: 0.0597213925793767 training accuracy: 1.0 test accuracy: 0.2857142857142857
Epoch: 1906 Avg_loss: 0.059139946661889556 training accuracy: 1.0 test accuracy: 0.2857142857142857
Epoch: 1907 Avg_loss: 0.05856198742985726 training accuracy: 1.0 test accuracy: 0.2857142857142857
Epoch: 19

Epoch: 1981 Avg_loss: 0.030093161202967168 training accuracy: 1.0 test accuracy: 0.23809523809523808
Epoch: 1982 Avg_loss: 0.029973371420055628 training accuracy: 1.0 test accuracy: 0.24489795918367346
Epoch: 1983 Avg_loss: 0.02980037936940789 training accuracy: 1.0 test accuracy: 0.23809523809523808
Epoch: 1984 Avg_loss: 0.029505628906190396 training accuracy: 1.0 test accuracy: 0.23809523809523808
Epoch: 1985 Avg_loss: 0.029251137096434832 training accuracy: 1.0 test accuracy: 0.23809523809523808
Epoch: 1986 Avg_loss: 0.02902843290939927 training accuracy: 1.0 test accuracy: 0.23809523809523808
Epoch: 1987 Avg_loss: 0.028786864038556813 training accuracy: 1.0 test accuracy: 0.23129251700680273
Epoch: 1988 Avg_loss: 0.028637092281132936 training accuracy: 1.0 test accuracy: 0.22448979591836735
Epoch: 1989 Avg_loss: 0.028448155149817467 training accuracy: 1.0 test accuracy: 0.22448979591836735
Epoch: 1990 Avg_loss: 0.028212430700659753 training accuracy: 1.0 test accuracy: 0.2312925170

In [13]:
model1, best_state1, best_acc1, loss1, acc1 = train_model(training_data, training_labels,
                                                             test_data, test_labels, 2000, 64, 16,
                                                             lr=0.0005, lamb=0.01, batch_size=32)

Epoch: 0 Avg_loss: 3.7881065130233766 training accuracy: 0.246875 test accuracy: 0.2925170068027211
Epoch: 1 Avg_loss: 3.2346574306488036 training accuracy: 0.39375 test accuracy: 0.3129251700680272
Epoch: 2 Avg_loss: 2.938950443267822 training accuracy: 0.45625 test accuracy: 0.2789115646258503
Epoch: 3 Avg_loss: 2.726707363128662 training accuracy: 0.4875 test accuracy: 0.30612244897959184
Epoch: 4 Avg_loss: 2.530256462097168 training accuracy: 0.525 test accuracy: 0.3333333333333333
Epoch: 5 Avg_loss: 2.3521998167037963 training accuracy: 0.546875 test accuracy: 0.2653061224489796
Epoch: 6 Avg_loss: 2.1843955755233764 training accuracy: 0.603125 test accuracy: 0.3129251700680272
Epoch: 7 Avg_loss: 2.022719645500183 training accuracy: 0.65 test accuracy: 0.2925170068027211
Epoch: 8 Avg_loss: 1.8797616243362427 training accuracy: 0.684375 test accuracy: 0.29931972789115646
Epoch: 9 Avg_loss: 1.7510489225387573 training accuracy: 0.709375 test accuracy: 0.3129251700680272
Epoch: 10 Avg

Epoch: 83 Avg_loss: 0.2846130609512329 training accuracy: 1.0 test accuracy: 0.2857142857142857
Epoch: 84 Avg_loss: 0.28290984630584715 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 85 Avg_loss: 0.28128315806388854 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 86 Avg_loss: 0.2797111958265305 training accuracy: 1.0 test accuracy: 0.29931972789115646
Epoch: 87 Avg_loss: 0.2782013535499573 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 88 Avg_loss: 0.27666805386543275 training accuracy: 1.0 test accuracy: 0.29931972789115646
Epoch: 89 Avg_loss: 0.2752715229988098 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 90 Avg_loss: 0.2738748282194138 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 91 Avg_loss: 0.27253676652908326 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 92 Avg_loss: 0.27115297615528106 training accuracy: 1.0 test accuracy: 0.2857142857142857
Epoch: 93 Avg_loss: 0.26980555951

Epoch: 166 Avg_loss: 0.25911361873149874 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 167 Avg_loss: 0.25734166204929354 training accuracy: 1.0 test accuracy: 0.29931972789115646
Epoch: 168 Avg_loss: 0.2557821929454803 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 169 Avg_loss: 0.2542300522327423 training accuracy: 1.0 test accuracy: 0.29931972789115646
Epoch: 170 Avg_loss: 0.2528239116072655 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 171 Avg_loss: 0.2514162346720695 training accuracy: 1.0 test accuracy: 0.2857142857142857
Epoch: 172 Avg_loss: 0.25015381872653963 training accuracy: 1.0 test accuracy: 0.2857142857142857
Epoch: 173 Avg_loss: 0.24889223873615265 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 174 Avg_loss: 0.24774073511362077 training accuracy: 1.0 test accuracy: 0.2857142857142857
Epoch: 175 Avg_loss: 0.24717580229043962 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 176 Avg_loss: 0

Epoch: 250 Avg_loss: 0.20348175317049028 training accuracy: 1.0 test accuracy: 0.29931972789115646
Epoch: 251 Avg_loss: 0.20317238718271255 training accuracy: 1.0 test accuracy: 0.29931972789115646
Epoch: 252 Avg_loss: 0.2027766451239586 training accuracy: 1.0 test accuracy: 0.29931972789115646
Epoch: 253 Avg_loss: 0.20247769504785537 training accuracy: 1.0 test accuracy: 0.29931972789115646
Epoch: 254 Avg_loss: 0.20203752368688582 training accuracy: 1.0 test accuracy: 0.29931972789115646
Epoch: 255 Avg_loss: 0.20175745636224746 training accuracy: 1.0 test accuracy: 0.29931972789115646
Epoch: 256 Avg_loss: 0.20128278136253358 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 257 Avg_loss: 0.20096681416034698 training accuracy: 1.0 test accuracy: 0.29931972789115646
Epoch: 258 Avg_loss: 0.20053593218326568 training accuracy: 1.0 test accuracy: 0.2925170068027211
Epoch: 259 Avg_loss: 0.20018824338912963 training accuracy: 1.0 test accuracy: 0.29931972789115646
Epoch: 260 Av

Epoch: 334 Avg_loss: 0.40523519814014436 training accuracy: 1.0 test accuracy: 0.3333333333333333
Epoch: 335 Avg_loss: 0.38783019185066225 training accuracy: 1.0 test accuracy: 0.32653061224489793
Epoch: 336 Avg_loss: 0.3725547671318054 training accuracy: 1.0 test accuracy: 0.3129251700680272
Epoch: 337 Avg_loss: 0.35892176926136016 training accuracy: 1.0 test accuracy: 0.30612244897959184
Epoch: 338 Avg_loss: 0.34698604643344877 training accuracy: 1.0 test accuracy: 0.30612244897959184
Epoch: 339 Avg_loss: 0.33624337911605834 training accuracy: 1.0 test accuracy: 0.30612244897959184
Epoch: 340 Avg_loss: 0.3267329096794128 training accuracy: 1.0 test accuracy: 0.30612244897959184
Epoch: 341 Avg_loss: 0.31804895102977754 training accuracy: 1.0 test accuracy: 0.30612244897959184
Epoch: 342 Avg_loss: 0.3104286640882492 training accuracy: 1.0 test accuracy: 0.3129251700680272
Epoch: 343 Avg_loss: 0.30331181585788725 training accuracy: 1.0 test accuracy: 0.3129251700680272
Epoch: 344 Avg_lo

...

Epoch: 1669 Avg_loss: 0.17088706493377687 training accuracy: 1.0 test accuracy: 0.3129251700680272
Epoch: 1670 Avg_loss: 0.17074906826019287 training accuracy: 1.0 test accuracy: 0.3129251700680272
Epoch: 1671 Avg_loss: 0.1706584632396698 training accuracy: 1.0 test accuracy: 0.3129251700680272
Epoch: 1672 Avg_loss: 0.17045178711414338 training accuracy: 1.0 test accuracy: 0.3129251700680272
Epoch: 1673 Avg_loss: 0.1706537589430809 training accuracy: 1.0 test accuracy: 0.3129251700680272
Epoch: 1674 Avg_loss: 0.17151785492897034 training accuracy: 1.0 test accuracy: 0.30612244897959184
Epoch: 1675 Avg_loss: 0.17105544358491898 training accuracy: 1.0 test accuracy: 0.3197278911564626
Epoch: 1676 Avg_loss: 0.17079183757305144 training accuracy: 1.0 test accuracy: 0.3129251700680272
Epoch: 1677 Avg_loss: 0.1706324115395546 training accuracy: 1.0 test accuracy: 0.32653061224489793
Epoch: 1678 Avg_loss: 0.17069289088249207 training accuracy: 1.0 test accuracy: 0.3129251700680272
Epoch: 1679

Epoch: 1750 Avg_loss: 0.2018576294183731 training accuracy: 1.0 test accuracy: 0.3401360544217687
Epoch: 1751 Avg_loss: 0.20123905837535858 training accuracy: 1.0 test accuracy: 0.3401360544217687
Epoch: 1752 Avg_loss: 0.20061311572790147 training accuracy: 1.0 test accuracy: 0.3401360544217687
Epoch: 1753 Avg_loss: 0.20003274232149124 training accuracy: 0.996875 test accuracy: 0.3469387755102041
Epoch: 1754 Avg_loss: 0.19944927245378494 training accuracy: 1.0 test accuracy: 0.3469387755102041
Epoch: 1755 Avg_loss: 0.1989117369055748 training accuracy: 1.0 test accuracy: 0.3401360544217687
Epoch: 1756 Avg_loss: 0.19841568619012834 training accuracy: 1.0 test accuracy: 0.3469387755102041
Epoch: 1757 Avg_loss: 0.19800253957509995 training accuracy: 1.0 test accuracy: 0.3401360544217687
Epoch: 1758 Avg_loss: 0.1974785178899765 training accuracy: 1.0 test accuracy: 0.3401360544217687
Epoch: 1759 Avg_loss: 0.19705894142389296 training accuracy: 1.0 test accuracy: 0.3469387755102041
Epoch: 1

Epoch: 1833 Avg_loss: 1.0229355096817017 training accuracy: 0.971875 test accuracy: 0.32653061224489793
Epoch: 1834 Avg_loss: 1.001638251543045 training accuracy: 0.95 test accuracy: 0.32653061224489793
Epoch: 1835 Avg_loss: 0.9318555414676666 training accuracy: 0.971875 test accuracy: 0.3197278911564626
Epoch: 1836 Avg_loss: 0.9093832790851593 training accuracy: 0.9875 test accuracy: 0.3401360544217687
Epoch: 1837 Avg_loss: 0.7884541153907776 training accuracy: 0.996875 test accuracy: 0.3877551020408163
Epoch: 1838 Avg_loss: 0.703834342956543 training accuracy: 0.996875 test accuracy: 0.3673469387755102
Epoch: 1839 Avg_loss: 0.6393381536006928 training accuracy: 0.996875 test accuracy: 0.35374149659863946
Epoch: 1840 Avg_loss: 0.5915814697742462 training accuracy: 0.996875 test accuracy: 0.3741496598639456
Epoch: 1841 Avg_loss: 0.5536887526512146 training accuracy: 0.996875 test accuracy: 0.3877551020408163
Epoch: 1842 Avg_loss: 0.5900746583938599 training accuracy: 0.996875 test accu

Epoch: 1916 Avg_loss: 0.19122399985790253 training accuracy: 1.0 test accuracy: 0.35374149659863946
Epoch: 1917 Avg_loss: 0.19098006337881088 training accuracy: 1.0 test accuracy: 0.3469387755102041
Epoch: 1918 Avg_loss: 0.19093565493822098 training accuracy: 1.0 test accuracy: 0.35374149659863946
Epoch: 1919 Avg_loss: 0.1906801000237465 training accuracy: 1.0 test accuracy: 0.3469387755102041
Epoch: 1920 Avg_loss: 0.1902298852801323 training accuracy: 1.0 test accuracy: 0.3469387755102041
Epoch: 1921 Avg_loss: 0.1897297665476799 training accuracy: 1.0 test accuracy: 0.3469387755102041
Epoch: 1922 Avg_loss: 0.18962065130472183 training accuracy: 1.0 test accuracy: 0.3469387755102041
Epoch: 1923 Avg_loss: 0.1895664244890213 training accuracy: 1.0 test accuracy: 0.35374149659863946
Epoch: 1924 Avg_loss: 0.18930122405290603 training accuracy: 1.0 test accuracy: 0.35374149659863946
Epoch: 1925 Avg_loss: 0.18902253210544587 training accuracy: 1.0 test accuracy: 0.3469387755102041
Epoch: 192

Epoch: 1999 Avg_loss: 0.2616805136203766 training accuracy: 1.0 test accuracy: 0.3333333333333333
best accuracy: 0.4013605442176871


In [ ]:
model1, best_state1, best_acc1, loss1, acc1 = train_model(training_data, training_labels,
                                                             test_data, test_labels, 2000, 128, 32,
                                                             lr=0.0005, lamb=0.005, batch_size=1)

Epoch: 0 Avg_loss: 1.8151039753109217 training accuracy: 0.221875 test accuracy: 0.22448979591836735
Epoch: 1 Avg_loss: 1.4822997376322746 training accuracy: 0.253125 test accuracy: 0.38095238095238093
Epoch: 2 Avg_loss: 1.449833869561553 training accuracy: 0.265625 test accuracy: 0.29931972789115646
Epoch: 3 Avg_loss: 1.424248882383108 training accuracy: 0.253125 test accuracy: 0.23129251700680273
Epoch: 4 Avg_loss: 1.521490865200758 training accuracy: 0.234375 test accuracy: 0.2108843537414966
Epoch: 5 Avg_loss: 1.4311349954456092 training accuracy: 0.259375 test accuracy: 0.22448979591836735
Epoch: 6 Avg_loss: 1.411452055722475 training accuracy: 0.253125 test accuracy: 0.2108843537414966
Epoch: 7 Avg_loss: 1.4431085471063851 training accuracy: 0.278125 test accuracy: 0.35374149659863946
Epoch: 8 Avg_loss: 1.5208096001297235 training accuracy: 0.2375 test accuracy: 0.2789115646258503
Epoch: 9 Avg_loss: 1.4246196579188108 training accuracy: 0.234375 test accuracy: 0.24489795918367346